Определение языка (language detection)
--------------------

* **Множество случаев** — тексты на разных языках
* **Множество классов** — языки

### Первый метод: частотные слова

Очень простой и неплохой по качеству метод. Сначала создаем частотный словарь для всех языков и берем самые частотные слова. После этого для каждого текста, который нам надо расклассифицировать, смотрим, частотных слов какого языка в нем больше - тот язык и выбираем.

Метод неплохо работает на текстах длиннее 50 слов и быстро имлементируется. 

В качестве корпусов и текстов для тестирования будем использовать статьи Википедии на разных языках. Скачать Википедию можно различными способами:

* Дампы википедии: https://dumps.wikimedia.org/backup-index.html

* wikiextractor: http://medialab.di.unipi.it/wiki/Wikipedia_Extractor

* annotated_wikiextractor: https://github.com/jodaiber/Annotated-WikiExtractor

* wikipedia: https://pypi.python.org/pypi/wikipedia/

#### Скачаем немного википедии для тестов
Воспользуемся пакетом *wikipedia*:

`pip install wikipedia`

In [83]:
def get_texts_for_lang(lang, n=10): # функция для скачивания статей из википедии
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
        except wikipedia.exceptions.WikipediaException:
            print('Skipping page {}'.format(page_name))
            continue

        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))

    return wiki_content

In [85]:
import wikipedia # скачиваем по 100 статей для каждого языка. Это может занять какое-то время (5-10 минут. как правило)

wiki_texts = {}
for lang in ('kk', 'uk', 'be', 'fr', 'ru'): # казахский в википедии — это kk,
                                      # украинский — uk, а белорусский — be
    wiki_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts[lang]))

kk 100


C:\Users\zhenya\A3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\zhenya\A3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Skipping page Михалович
Skipping page Хтонічний
Skipping page 14-та лінія
Skipping page 1279 (значення)
Skipping page Бакеєво
Skipping page Генріх IV
uk 94
Skipping page Асклепіяд
Skipping page Кулі
Skipping page Горка
Skipping page Брэст (значэнні)
be 96
Skipping page FGF
Skipping page Intemporel
Skipping page Nkolnguet
Skipping page Planète rebelle (éditeur)
fr 96
Skipping page Виноградов, Николай
Skipping page Родин, Борис
Skipping page Шаве
Skipping page ТФБ
Skipping page Томпсон, Дэвид
ru 95


In [86]:
print(wiki_texts['kk'][0]) # распечатаем пару текстов, чтобы убедиться, что все хорошо
print(wiki_texts['fr'][0])

IC 5173A
IC 5173A — Үндіс шоқжұлдызында орналасқан Sc (жинақы спиральді галактика) типті галактика.


 Ашылуы 
Бұл нысан Индекс каталогтың түпнұсқа басылымының тізіміне енеді. Сондай-ақ, бұл ғарыш объектісі өзге де зерттеулер мен каталогтарда кездескендіктен оның келесідей атаулары бар: PGC 68379, ESO 76-8A, AM 2210-693, Southern Integral-sign.


 Тағы қараңыз 
Мессье нысандарының тізімі
Жаңа жалпы каталог
Индекс каталог


 Сыртқы сілтемелер 
«Жаңа жалпы каталог» түпнұсқасындағы ағылшынша және французша мәліметтер
Қайта қаралған Жаңа жалпы каталогтағы мәліметтер (ағыл.)
SIMBAD базасындағы мәліметтер (ағыл.)
VizieR базасындағы мәліметтер (ағыл.)
NASA/IPAC Extragalactic Database базасындағы мәліметтер (ағыл.)
IC 5173A нысанына арналған жарияланымдар
Col de Perty
Le col de Perty à 1 302 mètres d'altitude relie Saint-Auban-sur-l'Ouvèze à l'ouest et Orpierre à l'est.


 Situation 
Il se situe dans le massif des Baronnies, à l'est de la montagne de la Clavelière, dans le département de la Dr

#### Считаем частотный список примерно так:

In [87]:
import codecs
import collections
import sys

def tokenize(text):
    return text.split(' ')

freqs_kk = collections.defaultdict(lambda: 0)

corpus = wiki_texts['kk']
for article in corpus:
    for word in tokenize(article.replace('\n', '').lower()):
        freqs_kk[word] += 1

for word in sorted(freqs_kk, key=lambda w: freqs_kk[w], reverse=True)[:50]:
    print('{}\t{}'.format(freqs_kk[word], word))

255	—
167	және
110	
87	су
79	бойынша
75	мен
72	дереккөздер
60	коды
59	өзен
54	-
53	сыртқы
53	сілтемелер
46	жер
45	бұл
44	үшін
43	мәліметтер
40	жылы
38	оның
38	саны
37	болып
37	ол
35	адамды
33	құрамына
33	«вконтакте»
32	тұрғындарының
32	жатқан
30	ресей
29	орналасқан.
29	алып
29	аумағы
29	өзеннің
28	км²
27	тағы
27	қазақстан
27	әлеуметтік
26	–
26	жайық
25	өзенінің
24	жылғы
24	басқа
23	жалпы
23	ресми
21	халық
21	бір
21	осы
21	да
21	дейін
21	алабы
21	жыл
20	сол


In [88]:
langs = ('kk', 'uk', 'be', 'fr', 'ru')

In [89]:
def freq_dict(langs, wiki_texts):
    freqs = {}
    for lang in langs:
        freqsl = collections.defaultdict(lambda: 0)
        corpus = wiki_texts[lang]
        for article in corpus:
            for word in tokenize(article.replace('\n', '').lower()):
                freqsl[word] += 1
        
        top50 = []
        for ngram in sorted(freqsl, key=lambda n: freqsl[n], reverse=True)[:50]:
            top50.append(ngram)
         
        freqs[lang] = top50
        
    return freqs


In [90]:
freqs = freq_dict(langs, wiki_texts)

#### Нужно сделать это для каждого языка и отфильтровать повторяющиеся

Теперь можно загружать готовые частотные словари и классифицировать тексты, просто считая количество слов в каждом.

### Второй метод: частотные символьные n-граммы

Создадим функцию, которая преобразовывает строку в массив n-грамм заданной длины.

In [91]:
from itertools import islice, tee

def make_ngrams(text):
    N = 3 # задаем длину n-граммы
    ngrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(text, N))))
    ngrams = [''.join(x) for x in ngrams]
    return ngrams

Теперь создадим частотные словари n-грамм аналогично первому методу.

In [92]:
freqs_fr2 = collections.defaultdict(lambda: 0)

corpus = wiki_texts['fr']
for article in corpus:
    for ngram in make_ngrams(article.replace('\n', '').lower()):
       # ngram = ngram.replace(' ', '')
        freqs_fr2[ngram] += 1

for ngram in sorted(freqs_fr2, key=lambda n: freqs_fr2[n], reverse=True)[:50]:
    print('{}\t{}'.format(freqs_fr2[ngram], ngram))

7961	 de
7464	es 
6212	de 
3927	e d
3826	le 
3730	 le
3332	nt 
3253	ent
3252	 la
3207	la 
3025	e l
2646	re 
2594	s d
2506	et 
2443	 co
2435	ion
2429	que
2252	on 
2219	 et
2197	les
2044	ne 
2032	en 
1950	 en
1904	 pa
1884	tio
1841	e p
1840	t d
1838	des
1824	ns 
1793	e s
1736	ue 
1714	 l'
1713	e c
1706	ur 
1655	par
1571	 à 
1547	e, 
1547	 qu
1528	est
1515	men
1509	 pr
1509	 po
1502	ant
1502	 un
1455	e e
1447	te 
1424	ati
1379	lle
1347	 du
1311	iqu


In [93]:
def freq_ngr(langs, wiki_texts):
    freqs_ngram = {}
    for lang in langs:
        freqs = collections.defaultdict(lambda: 0)
        corpus = wiki_texts[lang]
        for article in corpus:
            for ngram in make_ngrams(article.replace('\n', '').lower()):
     #   ngram = ngram.replace(' ', '')
                freqs[ngram] += 1
        
        top50 = []
        for ngram in sorted(freqs, key=lambda n: freqs[n], reverse=True)[:50]:
            top50.append(ngram)
         
        freqs_ngram[lang] = top50
        
    return freqs_ngram

In [94]:
freqs_ngram = freq_ngr(langs, wiki_texts)

In [95]:
print(freqs_ngram['fr'])

[' de', 'es ', 'de ', 'e d', 'le ', ' le', 'nt ', 'ent', ' la', 'la ', 'e l', 're ', 's d', 'et ', ' co', 'ion', 'que', 'on ', ' et', 'les', 'ne ', 'en ', ' en', ' pa', 'tio', 'e p', 't d', 'des', 'ns ', 'e s', 'ue ', " l'", 'e c', 'ur ', 'par', ' à ', 'e, ', ' qu', 'est', 'men', ' pr', ' po', 'ant', ' un', 'e e', 'te ', 'ati', 'lle', ' du', 'iqu']


#### Нужно сделать это для каждого языка и отфильтровать повторяющиеся

Теперь, как и в предыдущем методе, можно загружать готовые частотные словари n-грамм и классифицировать тексты, просто подсчитывая частотные n-граммы в каждом.

In [96]:
def predict_language_ngram(text, freqs_ngram):
    text_ngrams = make_ngrams(text.lower())
    
    lang2sim = {}
    
    for lang in ('kk', 'uk', 'be', 'fr', 'ru'):
        intersect = len(set(text_ngrams) & set(freqs_ngram[lang]))
        lang2sim[lang] = intersect
    
    return max(lang2sim, key=lambda x: lang2sim[x])

In [108]:
def predict_language_dict(text, freqs):
    text_words = tokenize(text.replace('\n', '').lower())
    
    lang2sim = {}
    
    for lang in ('kk', 'uk', 'be', 'fr', 'ru'):
        intersect = len(set(text_words) & set(freqs[lang]))
        lang2sim[lang] = intersect
    
    return max(lang2sim, key=lambda x: lang2sim[x])

In [99]:
a = predict_language(wiki_texts['kk'][0], freqs_ngram)

In [100]:
a

'kk'

In [101]:
from sklearn.metrics import classification_report, confusion_matrix

In [105]:
true_labels = []
predicted_labels = []

for lang in langs:
    for text in wiki_texts[lang]:
        true_labels.append(lang)
        predicted_labels.append(predict_language_ngram(text, freqs_ngram))

In [106]:
print(classification_report(true_labels, predicted_labels))

             precision    recall  f1-score   support

         be       1.00      0.98      0.99        96
         fr       1.00      1.00      1.00        96
         kk       1.00      1.00      1.00       100
         ru       1.00      1.00      1.00        95
         uk       0.98      1.00      0.99        94

avg / total       1.00      1.00      1.00       481



In [110]:
true_labels = []
predicted_labels = []

for lang in langs:
    for text in wiki_texts[lang]:
        true_labels.append(lang)
        predicted_labels.append(predict_language_dict(text, freqs))

In [111]:
print(classification_report(true_labels, predicted_labels))

             precision    recall  f1-score   support

         be       1.00      0.99      0.99        96
         fr       1.00      1.00      1.00        96
         kk       1.00      1.00      1.00       100
         ru       1.00      0.98      0.99        95
         uk       0.97      1.00      0.98        94

avg / total       0.99      0.99      0.99       481



Оба метода, и по словам, и по n-граммам, работают очень хорошо, но n-граммы немного лучше.

Проверим на новых текстах:

In [113]:
wiki_texts = {}
for lang in ('kk', 'uk', 'be', 'fr', 'ru'): # казахский в википедии — это kk,
                                      # украинский — uk, а белорусский — be
    wiki_texts[lang] = get_texts_for_lang(lang, 5)
    print(lang, len(wiki_texts[lang]))

kk 5
uk 5
be 5
fr 5
ru 5


In [114]:
true_labels = []
predicted_labels = []

for lang in langs:
    for text in wiki_texts[lang]:
        true_labels.append(lang)
        predicted_labels.append(predict_language_ngram(text, freqs_ngram))

In [115]:
print(classification_report(true_labels, predicted_labels))

             precision    recall  f1-score   support

         be       1.00      1.00      1.00         5
         fr       1.00      1.00      1.00         5
         kk       1.00      1.00      1.00         5
         ru       1.00      1.00      1.00         5
         uk       1.00      1.00      1.00         5

avg / total       1.00      1.00      1.00        25



In [116]:
true_labels = []
predicted_labels = []

for lang in langs:
    for text in wiki_texts[lang]:
        true_labels.append(lang)
        predicted_labels.append(predict_language_dict(text, freqs))

In [117]:
print(classification_report(true_labels, predicted_labels))

             precision    recall  f1-score   support

         be       1.00      1.00      1.00         5
         fr       1.00      1.00      1.00         5
         kk       1.00      1.00      1.00         5
         ru       1.00      1.00      1.00         5
         uk       1.00      1.00      1.00         5

avg / total       1.00      1.00      1.00        25

